In [ ]:
import datasets
from transformers import AutoTokenizer, BertTokenizerFast

from pathlib import Path
from typing import Generator

In [3]:

ds_path = '/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/corpus/dataset/dataset.parquet'
ds = datasets.Dataset.from_parquet(ds_path)

dsdict = ds.train_test_split(test_size=0.2)

Generating train split: 99 examples [00:00, 7216.73 examples/s]


In [6]:
print(dsdict)
# print(dsdict['train'][12]['code'])


DatasetDict({
    train: Dataset({
        features: ['disk', 'name', 'date', 'length', 'code'],
        num_rows: 79
    })
    test: Dataset({
        features: ['disk', 'name', 'date', 'length', 'code'],
        num_rows: 20
    })
})


In [19]:
def get_training_corpus(batch_size:int) -> Generator:
    return (
        dsdict["train"][i : i + batch_size]["code"]
        for i in range(0, len(dsdict["train"]), batch_size)
    )

In [21]:
for x in get_training_corpus(32):
    print(type(x), len(x), type(x[0]))

<class 'list'> 32 <class 'str'>
<class 'list'> 32 <class 'str'>
<class 'list'> 15 <class 'str'>


In [25]:
old_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

example = '10110 fori=256to511:pokef+i,peek(c+i):next:forz=0to31step8'

In [ ]:
# old_tokenizer.tokenize(example)

In [28]:
tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus(batch_size=32),
                                                  old_tokenizer.vocab_size)

In [29]:
tokenizer.tokenize(example)

['10110',
 'fori',
 '=',
 '256',
 '##to5',
 '##11',
 ':',
 'pokef',
 '+',
 'i',
 ',',
 'peek',
 '(',
 'c',
 '+',
 'i',
 ')',
 ':',
 'next',
 ':',
 'forz',
 '=',
 '0to3',
 '##1step',
 '##8']

In [32]:
savedir = Path("/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models")
savedir = savedir / f'{old_tokenizer.__class__.__qualname__}_trained'
savedir.mkdir(exist_ok=True)


tokenizer.save_pretrained(save_directory=savedir)

('/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models/BertTokenizerFast_trained/tokenizer_config.json',
 '/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models/BertTokenizerFast_trained/special_tokens_map.json',
 '/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models/BertTokenizerFast_trained/vocab.txt',
 '/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models/BertTokenizerFast_trained/added_tokens.json',
 '/Users/julian/Documents/3 - Bildung/31 - Studium/314 Universität Stuttgart/314.2 Semester 2/Projektarbeit/repository/inference/tokenization/models/BertTokenizerFast_trained/tokenizer.json')